In [3]:
!pip install tensorflow-addons==0.11.1

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
import numpy as np 
import matplotlib.pyplot as plt

import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.optimizers import Adam
import random

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

from sklearn.utils import class_weight as cw

In [5]:
tf.__version__

'2.3.0'

In [6]:
batch_size = 16
original_img_size = (256, 2600)
model_img_height = 256
model_img_width = 512
seed = 0
storage_dir = '../input/birdmel/train_img_final/train_img_final'
AUTOTUNE = tf.data.experimental.AUTOTUNE
crop_height = model_img_height
crop_width = model_img_width

In [7]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    storage_dir,
    validation_split=0.1,
    subset="training",
    seed=seed,
    image_size=original_img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    storage_dir,
    validation_split=0.1,
    subset="validation",
    seed=seed,
    image_size=original_img_size,
    batch_size=batch_size,
    label_mode='categorical',
)

Found 72956 files belonging to 265 classes.
Using 65661 files for training.
Found 72956 files belonging to 265 classes.
Using 7295 files for validation.


In [8]:
def build_model(num_classes, img_width=model_img_width, img_height=model_img_height):
    inputs = layers.Input(shape=(img_height, img_width, 3))
    model = EfficientNetB4(include_top=False, input_tensor=inputs)

    x = layers.GlobalMaxPooling2D(name="max_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.5
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    model.compile(
        optimizer=Adam(learning_rate=1e-3), 
        loss="categorical_crossentropy", 
        metrics=[tfa.metrics.F1Score(num_classes = class_num, average = 'micro')]
    )

    return model

In [9]:
random_crop_layer = preprocessing.RandomCrop(crop_height, crop_width)
random_contrast_layer = preprocessing.RandomContrast(factor=0.15)
random_translate_layer = preprocessing.RandomTranslation(height_factor=0, width_factor=0.15, fill_mode='constant')
rescale_layer = preprocessing.Rescaling(scale=1./255)

def resize(images, height=model_img_height, width=model_img_width):
    return tf.image.resize_with_crop_or_pad(images, target_height=height, target_width=width)

def random_mask(images, time_mask_length=40, freq_mask_length=20):
    images = tfa.image.random_cutout(
        images,
        mask_size=(model_img_height*3, time_mask_length)
    )
    
    images = tfa.image.random_cutout(
        images,
        mask_size=(freq_mask_length, model_img_width*3)
    )
    
    return images

def augment_image_train(images):    
    images = random_crop_layer(images)
    images = random_contrast_layer(images)
    images = random_mask(images)
    images = random_translate_layer(images)
    
    images = rescale_layer(images)
    images = resize(images)

    return images

def augment_image_test(images):
    images = rescale_layer(images)
    images = resize(images)
    
    return images

In [10]:
def prepare(ds, augment=False):
    # Use data augmentation only on the training set
    if augment:
        ds = ds.map(lambda x, y: (augment_image_train(x), y), num_parallel_calls=AUTOTUNE)
    else: 
        ds = ds.map(lambda x, y: (augment_image_test(x), y), num_parallel_calls=AUTOTUNE)

    # Use buffered prefecting on all datasets
    return ds.prefetch(buffer_size=AUTOTUNE)

In [11]:
class_num = len(train_ds.class_names)

In [12]:
train_ds = prepare(train_ds, augment=True)
val_ds = prepare(val_ds)

In [13]:
# image = next(iter(train_ds))[0][0, :, :, :]
# plt.imshow(image)

In [14]:
# image.shape

In [15]:
train_datagen = ImageDataGenerator(dtype=np.float16)

train_generator = train_datagen.flow_from_directory(
    directory=storage_dir,
    class_mode="categorical",
    target_size=original_img_size,
    batch_size=batch_size
)

class_weights = cw.compute_class_weight(
    'balanced',
    np.unique(train_generator.classes), 
    train_generator.classes
)

class_weights = {i : class_weights[i] for i in range(len(class_weights))}

del train_datagen
del train_generator

Found 72956 images belonging to 265 classes.


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 

In [16]:
net = build_model(class_num)

In [17]:
# net.load_weights( '../input/birdmel/efficientnet_b4_checkpoint_tf_2_3_0.h5')

In [18]:
model_check = ModelCheckpoint(
    'efn_b4_ckpt_tf_2_3_0.h5',
    monitor='val_f1_score', 
    verbose=0, 
    save_best_only=True, 
    save_weights_only=True,
    mode='max',
    period=1,
)

reduce_LR = ReduceLROnPlateau(
    monitor='val_f1_score',
    factor=0.2,
    patience=6, 
    min_lr=1e-5,
    cooldown=2
)

In [ ]:
net.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 100,
    class_weight = class_weights,
    callbacks = [reduce_LR, model_check]
)

Epoch 1/100
4104/4104 [==============================] - 2025s 494ms/step - loss: 6.7424 - f1_score: 0.0039 - val_loss: 5.7764 - val_f1_score: 0.0036
Epoch 2/100
 141/4104 [>.............................] - ETA: 31:06 - loss: 6.0544 - f1_score: 0.0066